# 필요한 라이브러리 불러오기

In [1]:
#pip install plaidml-keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install hyperopt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import plaidml
import plaidml.keras
plaidml.keras.install_backend()

In [4]:
# 데이터처리
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np # linear algebra
import math   
from datetime import datetime, date 

# 기타시스템
import warnings
warnings.filterwarnings('ignore')

# 데이터분리
from sklearn.model_selection import train_test_split

# 최적화
try : 
    from bayes_opt import BayesianOptimization 
except : 
    !pip install bayesian-optimization
    
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
    
# 정규화
from sklearn.preprocessing import MinMaxScaler


# 평가지표
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score

# 딥러닝
try : 
    import keras
except :
    !pip install keras
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM, GRU
from keras.layers import Input, Dense, Activation, Flatten, Dropout

print("Setup Complete")

Setup Complete


# 데이터 불러오기

In [5]:
dam = pd.read_csv("C:", encoding= 'cp949')
# dam = pd.read_csv("testdata.csv")
dam

,일시,댐수위(ELm),강우량(mm),유입량(㎡/sec),총방류량(㎡/sec),댐수위(ELm).1,강우량(mm).1,유입량(㎡/sec).1,총방류량(㎡/sec).1,댐수위(ELm).2,강우량(mm).2,유입량(㎡/sec).2,총방류량(㎡/sec).2,댐수위(ELm).3,강우량(mm).3,유입량(㎡/sec).3,총방류량(㎡/sec).3
0,2001-07-13 17:00,188.11,0.0,128.9,27.1,102.41,0.0,127.8,27.6,92.74,0.0,34.1,34.1,45.00,0.0,0.0,0.0
1,2001-07-13 18:00,188.13,0.0,106.3,27.1,102.43,0.0,133.5,27.7,92.74,0.0,34.1,34.1,45.00,0.0,0.0,0.0
2,2001-07-13 19:00,188.14,0.0,100.9,27.1,102.44,0.0,116.9,27.7,92.74,0.0,34.1,34.1,45.00,0.0,0.0,0.0
3,2001-07-13 20:00,188.16,0.0,101,27.1,102.46,0.0,122.6,27.7,92.74,0.0,34.1,34.1,45.00,0.0,0.0,0.0
4,2001-07-13 21:00,188.17,0.0,95.3,26.6,102.47,0.0,111.5,27.7,92.74,0.0,34.1,34.1,45.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175652,2021-07-28 12:00,185.02,0.0,0,36.2,102.86,0.0,20.6,20.6,100.10,0.0,9.5,9.5,76.99,0.0,4.8,4.8
175653,2021-07-28 13:00,185.01,0.0,0,36.2,102.86,0.0,20.6,20.6,100.11,0.0,26.1,9.5,76.99,0.0,4.8,4.8
175654,2021-07-28 14:00,185.00,0.0,0,36.2,102.86,0.0,21.3,21.3,100.11,0.0,9.5,9.5,76.99,0.0,4.8,4.8
175655,2021-07-28 15:00,185.00,0.0,36.1,36.1,102.85,0.0,0,21.7,100.11,0.0,9.4,9.4,76.99,0.0,4.8,4.8


## 데이터 분리

In [29]:
def datasplit_ts(raw, Y_colname, X_colname, criteria):
    raw_train = raw.loc[raw.index < criteria,:]
    raw_test = raw.loc[raw.index >= criteria,:]
    Y_train = raw_train[Y_colname]
    X_train = raw_train[X_colname]
    Y_test = raw_test[Y_colname]
    X_test = raw_test[X_colname]
    print('Train_size:', raw_train.shape, 'Test_size:', raw_test.shape)
    print('X_train:', X_train.shape, 'Y_train:', Y_train.shape)
    print('X_test:', X_test.shape, 'Y_test:', Y_test.shape)
    return X_train, X_test, Y_train, Y_test

# 파라미터 찾기제작

In [30]:
#모델에 필요한 하이퍼파라미터의 범위를 설정하여 설정한 범위 안에서 최적의 하이퍼파라미터를 찾을 수 있음
space={'batch_size': hp.quniform("batch_size", 64, 385, 64), #(최소범위, 최대범위+1, 범위 안에서의 구간 설정)
       'sequence': hp.quniform ('sequence', 24, 73, 24),  #예를 들어 (24,73,24)일 경우, 24에서 72까지의 범위에서 24구간으로 24, 48, 72 중 최적 값을 찾음
       'dropout_ratio': hp.quniform('dropout_ratio', 0, 0.6, 0.1),
       'epoch' : hp.quniform('epoch', 10, 51, 10)
      }

In [31]:
#아래 코드에서 사용하려는 모델의 구조를 설정하여, 모델에 맞는 최적화를 실행

def hyperparameter_tuning(space):
    
    sequence = int(space['sequence'])
    
    Y_colname = ['water_level2']
    X_remove = ['date', 'datetime','water_level','water_level3','water_level4']
    X_colname = [x for x in features.columns if x not in Y_colname+X_remove]
    X_train, X_test, Y_train, Y_test = datasplit_ts(raw_nfe, Y_colname, X_colname, '2017-01-01')
    # Parameters
    scaler_X_tr = MinMaxScaler()
    scaler_Y_tr = MinMaxScaler()

    # Feature Engineering
    ## 스케일링
    X_train_scaled = scaler_X_tr.fit_transform(X_train)
    Y_train_scaled = scaler_Y_tr.fit_transform(Y_train)
    X_test_scaled = scaler_X_tr.transform(X_test)
    Y_test_scaled = scaler_Y_tr.transform(Y_test)
    
    ## X / Y 분리
    X_train, Y_train = [], []
    for index in range(len(X_train_scaled) - sequence):
        X_train.append(np.array(X_train_scaled[index: index + sequence]))
        Y_train.append(np.ravel(Y_train_scaled[index + sequence:index + sequence + 1]))
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_test, Y_test = [], []
    for index in range(len(X_test_scaled) - sequence):
        X_test.append(np.array(X_test_scaled[index: index + sequence]))
        Y_test.append(np.ravel(Y_test_scaled[index + sequence:index + sequence + 1]))  
    X_test, Y_test = np.array(X_test), np.array(Y_test) 

    # Retype and Reshape
    X_train = X_train.reshape(X_train.shape[0], sequence, -1)
    X_test = X_test.reshape(X_test.shape[0], sequence, -1)
#     print('X_train:', X_train.shape, 'Y_train:', Y_train.shape)
#     print('X_test:', X_test.shape, 'Y_test:', Y_test.shape)

    evaluation = [(X_train, Y_train), (X_test, Y_test)]
    
    # GRU
    model = Sequential()
    model.add(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
    model.add(Dropout(space['dropout_ratio'])) 
    model.add(GRU(64, return_sequences=False, activation="relu"))
    model.add(Dropout(space['dropout_ratio'])) 
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    # model.summary()
    model_fit = model.fit(X_train, Y_train, 
                              batch_size=int(space['batch_size']), epochs= int(space['epoch']),
                              verbose=0)
    pred = model.predict(X_test)
    mse= mean_squared_error(Y_test, pred)
    r2 = r2_score(Y_test, pred)
    
    # 평가 방식 선정
    return {'loss':mse, 'status': STATUS_OK, 'model': model}
        
    

In [32]:
trials = Trials()

#max_evals가 최적화 수행을 몇 번 진행할 것인지를 결정하는데, 많이 하면 할수록 더 정확성이 높아질 거임
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=4,  #(4번)
            trials=trials)

best['batch_size'] = best['batch_size']
best['sequence'] = best['sequence']
best['dropout_ratio'] = best['dropout_ratio']
best['epoch'] = best['epoch']
print (best)

Train_size:                                          
(135607, 22)                                         
Test_size:                                           
(40073, 22)                                          
X_train:                                             
(135607, 16)                                         
Y_train:                                             
(135607, 1)                                          
X_test:                                              
(40073, 16)                                          
Y_test:                                              
(40073, 1)                                           
  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

INFO:plaidml:Opening device "opencl_nvidia_geforce_gtx_1660_super.0"
INFO:plaidml:Analyzing Ops: 5862 of 10725 operations complete
INFO:plaidml:Analyzing Ops: 4746 of 10725 operations complete


Train_size:                                                                          
(135607, 22)                                                                         
Test_size:                                                                           
(40073, 22)                                                                          
X_train:                                                                             
(135607, 16)                                                                         
Y_train:                                                                             
(135607, 1)                                                                          
X_test:                                                                              
(40073, 16)                                                                          
Y_test:                                                                              
(40073, 1)                                            

INFO:plaidml:Analyzing Ops: 6709 of 7222 operations complete
INFO:plaidml:Analyzing Ops: 6349 of 7222 operations complete


Train_size:                                                                          
(135607, 22)                                                                       
Test_size:                                                                         
(40073, 22)                                                                        
X_train:                                                                           
(135607, 16)                                                                       
Y_train:                                                                           
(135607, 1)                                                                        
X_test:                                                                            
(40073, 16)                                                                        
Y_test:                                                                            
(40073, 1)                                                                

INFO:plaidml:Analyzing Ops: 5066 of 7222 operations complete
INFO:plaidml:Analyzing Ops: 4226 of 7222 operations complete


Train_size:                                                                        
(135607, 22)                                                                      
Test_size:                                                                        
(40073, 22)                                                                       
X_train:                                                                          
(135607, 16)                                                                      
Y_train:                                                                          
(135607, 1)                                                                       
X_test:                                                                           
(40073, 16)                                                                       
Y_test:                                                                           
(40073, 1)                                                                        
 75

INFO:plaidml:Analyzing Ops: 5866 of 7222 operations complete
INFO:plaidml:Analyzing Ops: 5881 of 7222 operations complete
ERROR:plaidml:unable to run OpenCL kernel kernel_c18_sdk_407: CL_MEM_OBJECT_ALLOCATION_FAILURE


100%|██████████| 4/4 [54:00<00:00, 810.10s/trial, best loss: 0.023695736450656674]
{'batch_size': 320.0, 'dropout_ratio': 0.2, 'epoch': 50.0, 'sequence': 72.0}
